<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#!pip install onnx onnxruntime


In [2]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name_A = "A"
input_name_B = "B"
output_name = "Output"

#-------------------------------------------------------------------------------
# 2-rank input tensor for A and B
#-------------------------------------------------------------------------------

# Create 2-rank input tensors for A and B
input_tensor_A = onnx.helper.make_tensor_value_info(input_name_A, onnx.TensorProto.FLOAT, [None, None])
input_tensor_B = onnx.helper.make_tensor_value_info(input_name_B, onnx.TensorProto.FLOAT, [None, None])

# Create output tensor for A^B
output_tensor = onnx.helper.make_tensor_value_info(output_name, onnx.TensorProto.FLOAT, [None, None])

# Define Power node: Y = A^B
power_node = onnx.helper.make_node("Pow", [input_name_A, input_name_B], [output_name])

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[power_node],
    name="Power",
    inputs=[input_tensor_A, input_tensor_B],
    outputs=[output_tensor]
)

# Create the ONNX model (Pow is available in opset 13)
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 13)]) # Explicitly set opset to 13
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "power.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("power.onnx")

def do_power(A, B):
    # Run inference
    output = session.run(None, {input_name_A: A, input_name_B: B})

    A_f = (np.array2string(A, separator=',', max_line_width=np.inf).replace('\n', ''))
    B_f = (np.array2string(B, separator=',', max_line_width=np.inf).replace('\n', ''))
    O_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))

    # Display results
    print(f"A={A_f}")
    print(f"B={B_f}")
    print(f"Result: A^B={O_f}")

np.set_printoptions(precision=None, floatmode='fixed')

#--------------------------------------------------------------------
# Values for A and B (as per your test set)
#--------------------------------------------------------------------

values = [float("-inf"), -10.0, -1/2, -1/3, -0.0, 0.0, 1/3,1/2, 10.0, float("inf"), float("nan")]

# Create a matrix where each row is a combination of A and B values
# The matrix will have A as the first dimension and B as the second dimension

A_vals = np.array(values).reshape(-1, 1)  # Reshape for broadcasting
B_vals = np.array(values).reshape(1, -1)  # Reshape for broadcasting

# Create a full matrix with all combinations of A and B
X_A = np.tile(A_vals, (1, len(values)))  # Repeat A values in rows
X_B = np.tile(B_vals, (len(values), 1))  # Repeat B values in columns

# Convert matrices to float32 to avoid type mismatch
X_A = X_A.astype(np.float32)
X_B = X_B.astype(np.float32)

# Now, X_A and X_B are matrices with A and B values respectively
print("Matrix of A values:")
print(X_A)

print("\nMatrix of B values:")
print(X_B)

# Run the power operation (A^B) on the matrix
do_power(X_A, X_B)


Matrix of A values:
[[        -inf         -inf         -inf         -inf         -inf
          -inf         -inf         -inf         -inf         -inf
          -inf]
 [-10.00000000 -10.00000000 -10.00000000 -10.00000000 -10.00000000
  -10.00000000 -10.00000000 -10.00000000 -10.00000000 -10.00000000
  -10.00000000]
 [ -0.50000000  -0.50000000  -0.50000000  -0.50000000  -0.50000000
   -0.50000000  -0.50000000  -0.50000000  -0.50000000  -0.50000000
   -0.50000000]
 [ -0.33333334  -0.33333334  -0.33333334  -0.33333334  -0.33333334
   -0.33333334  -0.33333334  -0.33333334  -0.33333334  -0.33333334
   -0.33333334]
 [ -0.00000000  -0.00000000  -0.00000000  -0.00000000  -0.00000000
   -0.00000000  -0.00000000  -0.00000000  -0.00000000  -0.00000000
   -0.00000000]
 [  0.00000000   0.00000000   0.00000000   0.00000000   0.00000000
    0.00000000   0.00000000   0.00000000   0.00000000   0.00000000
    0.00000000]
 [  0.33333334   0.33333334   0.33333334   0.33333334   0.33333334
    0.3333333

In [ ]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "A"
pow_output_name = "PowOutput"

#-------------------------------------------------------------------------------
# 2-rank input tensor
#-------------------------------------------------------------------------------

# Create 2-rank input tensor for A and B
input_tensor_A = onnx.helper.make_tensor_value_info(input_name, onnx.TensorProto.FLOAT, [None, None])
input_tensor_B = onnx.helper.make_tensor_value_info("B", onnx.TensorProto.FLOAT, [None, None])

# Create output tensor (final result after POW operation)
pow_output = onnx.helper.make_tensor_value_info(pow_output_name, onnx.TensorProto.FLOAT, [None, None])

# Define POW node: Y = A^B
pow_node = onnx.helper.make_node("Pow", [input_name, "B"], [pow_output_name])

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[pow_node],
    name="Pow",
    inputs=[input_tensor_A, input_tensor_B],
    outputs=[pow_output]
)

# Create the ONNX model (POW is available in opset 13)
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 13)])  # Explicitly set opset to 13
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "pow.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("pow.onnx")

def do_pow(x_A, x_B):
    # Run inference
    output = session.run(None, {input_name: x_A, "B": x_B})

    x_A_f = np.array2string(x_A, separator=',', max_line_width=np.inf).replace('\n', '')
    x_B_f = np.array2string(x_B, separator=',', max_line_width=np.inf).replace('\n', '')
    o_f = np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '')

    # Display results
    print(f"A={x_A_f}")
    print(f"B={x_B_f}")
    print(f"Result: A^B={o_f}")

np.set_printoptions(precision=None, floatmode='fixed')


#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

# Case N1: 2-rank tensor (float32)
A = np.array([[9.0, 4.0, 16.0, 8.0, 2.0]], dtype=np.float32)
B = np.array([[2.0, 2.5, 0.5, 1/3, 1.5]], dtype=np.float32)

do_pow(A, B)

#--------------------------------------------------------------------
# Non nominal cases (nan, -inf, -0, 0, negative values)
#--------------------------------------------------------------------

v = [1.0, 0.0, -0.0, -1.0, float("inf"), float("-inf"), float("nan")]

for x_A_val in v:
    for x_B_val in v:
        A_np = np.array([[x_A_val]], dtype=np.float32)
        B_np = np.array([[x_B_val]], dtype=np.float32)
        do_pow(A_np, B_np)

#--------------------------------------------------------------------
# Examples from documentation
#--------------------------------------------------------------------

print("\n## Example 1")

# Example 1
# A = [9, 4, 16, 8, 2]
# B = [2, 2.5, 0.5, 1/3, 3/2]
x_A_example_1 = np.array([[9.0, 4.0, 16.0, 8.0, 2.0]], dtype=np.float32)
x_B_example_1 = np.array([[2.0, 2.5, 0.5, 1/3, 1.5]], dtype=np.float32)
do_pow(x_A_example_1, x_B_example_1)

print("\n## Example 2")

# Example 2
# A = [0, 0, 5, -5, -25, -8]
# B = [0, 2, 0, 0, 3/5, 1/3]
x_A_example_2 = np.array([[0.0, 0.0, 5.0, -5.0, -25.0, -8.0]], dtype=np.float32)
x_B_example_2 = np.array([[0.0, 2.0, 0.0, 0.0, 3/5, 1/3]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)



# Example 3
x_A_example_2 = np.array([[-8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0, -8.0]], dtype=np.float32)
x_B_example_2 = np.array([[0.0, 2.0, 3.0, -2.0, -3.0, 2.0, 3.0, 2.5, -1/3, -1/5, -1/6, 1/3, 1/5, 1/6, -0.3333333, -0.33333333, -0.333333333, -0.3333333333, -0.33333333333, -0.3333333, -0.33333333, 0.333333333, 0.3333333333, 0.33333333333]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)


x_A_example_2 = np.array([[-8.0, -8.0, -8.0]], dtype=np.float32)
x_B_example_2 = np.array([[2.0000001, 2.00000011, 2.00000012]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)






A=[[ 9.00000000, 4.00000000,16.00000000, 8.00000000, 2.00000000]]
B=[[2.00000000,2.50000000,0.50000000,0.33333334,1.50000000]]
Result: A^B=[[81.00000000,32.00000000, 4.00000000, 2.00000000, 2.82842708]]
A=[[1.00000000]]
B=[[1.00000000]]
Result: A^B=[[1.00000000]]
A=[[1.00000000]]
B=[[0.00000000]]
Result: A^B=[[1.00000000]]
A=[[1.00000000]]
B=[[-0.00000000]]
Result: A^B=[[1.00000000]]
A=[[1.00000000]]
B=[[-1.00000000]]
Result: A^B=[[1.00000000]]
A=[[1.00000000]]
B=[[inf]]
Result: A^B=[[1.00000000]]
A=[[1.00000000]]
B=[[-inf]]
Result: A^B=[[1.00000000]]
A=[[1.00000000]]
B=[[nan]]
Result: A^B=[[1.00000000]]
A=[[0.00000000]]
B=[[1.00000000]]
Result: A^B=[[0.00000000]]
A=[[0.00000000]]
B=[[0.00000000]]
Result: A^B=[[1.00000000]]
A=[[0.00000000]]
B=[[-0.00000000]]
Result: A^B=[[1.00000000]]
A=[[0.00000000]]
B=[[-1.00000000]]
Result: A^B=[[inf]]
A=[[0.00000000]]
B=[[inf]]
Result: A^B=[[0.00000000]]
A=[[0.00000000]]
B=[[-inf]]
Result: A^B=[[inf]]
A=[[0.00000000]]
B=[[nan]]
Result: A^B=[[nan]]


'|pow(+Inf, y) | returns +0 for y negative, and +Inf for y positive.|'

In [61]:



print("|pow(±0, y) | returns ±Inf for y a negative odd integer.|")
x_A_example_2 = np.array([[0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0]], dtype=np.float32)
x_B_example_2 = np.array([[-1, -3, -5, -7, -1, -3, -5, -7]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(±0, y) | returns +Inf for y negative and not an odd integer.|")
x_A_example_2 = np.array([[0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0]], dtype=np.float32)
x_B_example_2 = np.array([[-2, -4, -6, -8, -2, -4, -6, -8]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(±0, y) | returns ±0 for y a positive odd integer.|")
x_A_example_2 = np.array([[0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0]], dtype=np.float32)
x_B_example_2 = np.array([[1, 3, 5, 7, 1, 3, 5, 7]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(±0, y) | returns +0 for y positive and not an odd integer.|")
x_A_example_2 = np.array([[0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0]], dtype=np.float32)
x_B_example_2 = np.array([[2, 4, 6, 8, 2, 4, 6, 8]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(-1, ±Inf) | returns 1.|")
x_A_example_2 = np.array([[-1.0, -1.0]], dtype=np.float32)
x_B_example_2 = np.array([[float("-inf"), float("inf")]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(+1, y) | returns 1 for any y, even a NaN.|")
x_A_example_2 = np.array([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]], dtype=np.float32)
x_B_example_2 = np.array([[float("NaN"), float("-inf"), -3, -2, -1/3, -1/2, -0, 0, 1/2, 1/3,2,3,float("inf")]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(x, ±0) | returns 1 for any x, even a NaN.|")
x_A_example_2 = np.array([[float("NaN"), float("-inf"), -3, -2, -1/3, -1/2, -0, 0, 1/2, 1/3,2,3,float("inf"),float("NaN"), float("-inf"), -3, -2, -1/3, -1/2, -0, 0, 1/2, 1/3,2,3,float("inf")]], dtype=np.float32)
x_B_example_2 = np.array([[-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,-0,0,0,0,0,0,0,0,0,0,0,0,0,0]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(x, y) | returns NaN for finite negative x and finite non-integer y.(-0 is not negative)|")
x_A_example_2 = np.array([[-3, -2, -1/3, -1/2]], dtype=np.float32)
x_B_example_2 = np.array([[-0.1, -1/3, -1/2, -1.1]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")


print("|pow(x, -Inf) | returns +Inf for 0 < abs(x) < 1, and +0 for abs(x) > 1.|")
x_A_example_2 = np.array([[0.1, 1/3, 1/2, 0.999999, 1.0000001, 1.1, 2.0, 10.5,-1.0000001, -1.1, -2.0, -10.5]], dtype=np.float32)
x_B_example_2 = np.array([[float("-inf"), float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf")]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")



print("|pow(x, +Inf) | returns +0 for 0 < abs(x) < 1, and +Inf for abs(x) > 1.|")
x_A_example_2 = np.array([[0.1, 1/3, 1/2, 0.999999, 1.0000001, 1.1, 2.0, 10.5, -1.1, -2.0, -10.5]], dtype=np.float32)
x_B_example_2 = np.array([[float("inf"),float("inf"),float("inf"),float("inf"), float("inf"),float("inf"),float("inf"),float("inf"),float("inf"),float("inf"),float("inf"),]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")


print("|pow(-Inf, y) | returns −0 for y a negative odd integer.|")
x_A_example_2 = np.array([[float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf")]], dtype=np.float32)
x_B_example_2 = np.array([[-1, -3, -5, -7, -9, -11, -13, -15]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(-Inf, y) | returns +0 for y negative and not an odd integer.|")
x_A_example_2 = np.array([[float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf")]], dtype=np.float32)
x_B_example_2 = np.array([[-2, -4, -6, -8, -10, -12, -14, -16]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(-Inf, y) | returns −Inf for y a positive odd integer.|")
x_A_example_2 = np.array([[float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf")]], dtype=np.float32)
x_B_example_2 = np.array([[1, 3, 5, 7, 9, 11, 13, 15]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(-Inf, y) | returns +Inf for y positive and not an odd integer.|")
x_A_example_2 = np.array([[float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf"),float("-inf")]], dtype=np.float32)
x_B_example_2 = np.array([[2, 4, 6, 8, 10, 12, 14, 16]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")

print("|pow(+Inf, y) | returns +0 for y negative, and +Inf for y positive.|")
x_A_example_2 = np.array([[float("inf"),float("inf"),float("inf"),float("inf"),float("inf")]], dtype=np.float32)
x_B_example_2 = np.array([[-1, -3, -5, -7, float("inf")]], dtype=np.float32)
do_pow(x_A_example_2, x_B_example_2)
print("---")


|pow(±0, y) | returns ±Inf for y a negative odd integer.|
A=[[ 0.00000000, 0.00000000, 0.00000000, 0.00000000,-0.00000000,-0.00000000,-0.00000000,-0.00000000]]
B=[[-1.00000000,-3.00000000,-5.00000000,-7.00000000,-1.00000000,-3.00000000,-5.00000000,-7.00000000]]
Result: A^B=[[ inf, inf, inf, inf,-inf,-inf,-inf,-inf]]
---
|pow(±0, y) | returns +Inf for y negative and not an odd integer.|
A=[[ 0.00000000, 0.00000000, 0.00000000, 0.00000000,-0.00000000,-0.00000000,-0.00000000,-0.00000000]]
B=[[-2.00000000,-4.00000000,-6.00000000,-8.00000000,-2.00000000,-4.00000000,-6.00000000,-8.00000000]]
Result: A^B=[[inf,inf,inf,inf,inf,inf,inf,inf]]
---
|pow(±0, y) | returns ±0 for y a positive odd integer.|
A=[[ 0.00000000, 0.00000000, 0.00000000, 0.00000000,-0.00000000,-0.00000000,-0.00000000,-0.00000000]]
B=[[1.00000000,3.00000000,5.00000000,7.00000000,1.00000000,3.00000000,5.00000000,7.00000000]]
Result: A^B=[[ 0.00000000, 0.00000000, 0.00000000, 0.00000000,-0.00000000,-0.00000000,-0.00000000,-0.00